In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
from time import time
import datetime
from catboost import CatBoostClassifier
import xgboost
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier 
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
from category_encoders.ordinal import OrdinalEncoder
from scipy.stats import mode
import json
import psutil
import os
from copy import deepcopy

# tqdm.notebook.tqdm

def cpu_stats():
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return 'memory GB:' + str(np.round(memory_use, 2))
# from numba import jit

import IPython

def display(*dfs):
    for df in dfs:
        IPython.display.display(df)
cpu_stats()

In [ ]:
train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')

In [ ]:
from sklearn.metrics import confusion_matrix
def qwk(act,pred, n=4, hist_range=(0,3)):
    
    O = confusion_matrix(act,pred)
    O = np.divide(O,np.sum(O))
    
    W = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            W[i][j] = ((i-j)**2)/((n-1)**2)
            
    act_hist = np.histogram(act,bins=n,range=hist_range)[0]
    prd_hist = np.histogram(pred,bins=n,range=hist_range)[0]
    
    E = np.outer(act_hist,prd_hist)
    E = np.divide(E,np.sum(E))
    
    num = np.sum(np.multiply(W,O))
    den = np.sum(np.multiply(W,E))
        
    return 1-np.divide(num,den)

In [ ]:
def titles_to_map():
    titles_dict = {}
    titles_dict['Welcome to Lost Lagoon!'] = 'WLL'

    # CRYSTALCAVES
    titles_dict['Chicken Balancer (Activity)'] = 'C_AC_CB'
    titles_dict['Egg Dropper (Activity)'] = 'C_AC_ED'

    titles_dict['Cart Balancer (Assessment)'] = 'C_AS_CB'
    titles_dict['Chest Sorter (Assessment)'] = 'C_AS_CS'

    titles_dict['Balancing Act'] = 'C_C_BA'
    titles_dict['Crystal Caves - Level 1'] = 'C_C_CC1'
    titles_dict['Crystal Caves - Level 2'] = 'C_C_CC2'
    titles_dict['Crystal Caves - Level 3'] = 'C_C_CC3'
    titles_dict['Heavy, Heavier, Heaviest'] = 'C_C_HHH'
    titles_dict['Honey Cake'] = 'C_C_HC'
    titles_dict['Lifting Heavy Things'] = 'C_C_LHT'

    titles_dict['Chow Time'] = 'C_G_CT'
    titles_dict['Happy Camel'] = 'C_G_HC'
    titles_dict['Leaf Leader'] = 'C_G_LL'
    titles_dict['Pan Balance'] = 'C_G_PB'

    #MAGMAPEAK
    titles_dict['Bottle Filler (Activity)'] = 'M_AC_BF'
    titles_dict['Sandcastle Builder (Activity)'] = 'M_AC_SB'
    titles_dict['Watering Hole (Activity)'] = 'M_AC_WH'

    titles_dict['Cauldron Filler (Assessment)'] = 'M_AS_CF'

    titles_dict['Magma Peak - Level 1'] = 'M_C_MPL1'
    titles_dict['Magma Peak - Level 2'] = 'M_C_MPL2'
    titles_dict['Slop Problem'] = 'M_C_SP'

    titles_dict['Bubble Bath'] = 'M_G_BB'
    titles_dict['Dino Dive'] = 'M_G_DD'
    titles_dict['Dino Drink'] = 'M_G_DDR'
    titles_dict['Scrub-A-Dub'] = 'M_G_SAD'

    #TREETOPCITY
    titles_dict['Bug Measurer (Activity)'] = 'T_AC_BM'
    titles_dict['Fireworks (Activity)'] = 'T_AC_F'
    titles_dict['Flower Waterer (Activity)'] = 'T_AC_FW'

    titles_dict['Bird Measurer (Assessment)'] = 'T_AS_BM'
    titles_dict['Mushroom Sorter (Assessment)'] = 'T_AS_MS'

    titles_dict['12 Monkeys'] = 'T_C_12M'
    titles_dict['Costume Box'] = 'T_C_CB'
    titles_dict['Ordering Spheres'] = 'T_C_OS'
    titles_dict["Pirate's Tale"] = 'T_C_PT'
    titles_dict['Rulers'] = 'T_C_R'
    titles_dict['Treasure Map'] = 'T_C_TM'
    titles_dict['Tree Top City - Level 1'] = 'T_C_TTC1'
    titles_dict['Tree Top City - Level 2'] = 'T_C_TTC2'
    titles_dict['Tree Top City - Level 3'] = 'T_C_TTC3'

    titles_dict['Air Show'] = 'T_G_AS'
    titles_dict['All Star Sorting'] = 'T_G_ASS'
    titles_dict['Crystals Rule'] = 'T_G_CR'
    
    return titles_dict

In [ ]:
# Generate simplified column name for all the titles
titles_dict = titles_to_map()
train['title'] = train['title'].map(titles_dict)
test['title'] = test['title'].map(titles_dict)

In [ ]:
def get_correct_flag(event_data):
    if 'true' in event_data:
        return 1
    elif 'false' in event_data:
        return 0
    else:
        return -1

In [ ]:
train['correct_flag'] = train.event_data.apply(lambda event_data: get_correct_flag(event_data))
test['correct_flag'] = test.event_data.apply(lambda event_data: get_correct_flag(event_data))

In [ ]:
# Initialise all the feature list
# Initiate feature list for all the assessment
def generate_T_AS_MS_features():
    
    title_name = 'T_AS_MS'
    T_AS_MS_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "win", "miss", "per_win", "count_result", "acc", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed"
                   ] 

    T_AS_MS_agg_list = []
    [T_AS_MS_agg_list.append(title_name +'_mean_'+ key) for key in  T_AS_MS_list]
    [T_AS_MS_agg_list.append(title_name +'_mode_'+ key) for key in  T_AS_MS_list]
    [T_AS_MS_agg_list.append(title_name +'_std_'+ key) for key in  T_AS_MS_list]
    
    return T_AS_MS_agg_list

def generate_T_AS_BM_features():
    
    title_name = 'T_AS_BM' 
    T_AS_BM_list = ["correct_drag", "incorrect_drag", "per_correct_drag", "count_drag", \
                    "drag_win", "drag_miss", "per_drag_win", "count_drag_result", \
                    "correct_hat", "incorrect_hat", "per_correct_hat", "count_hat", \
                    "hat_win", "hat_miss", "per_hat_win", "count_hat_result", \
                    "acc", "extra_clicks", "help_count", "game_time", "event_count","speed" \
                   ]
    
    T_AS_BM_agg_list = []
    [T_AS_BM_agg_list.append(title_name +'_mean_'+ key) for key in  T_AS_BM_list]
    [T_AS_BM_agg_list.append(title_name +'_mode_'+ key) for key in  T_AS_BM_list]
    [T_AS_BM_agg_list.append(title_name +'_std_'+ key) for key in  T_AS_BM_list]
    
    return T_AS_BM_agg_list

def generate_M_AS_CF_features():
    
    title_name = 'M_AS_CF' 
    M_AS_CF_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "win", "miss", "per_win", "count_result", "acc", \
                    "correct_bucket","incorrect_bucket", "per_bucket_click", "count_bucket_click", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    M_AS_CF_agg_list = []
    [M_AS_CF_agg_list.append(title_name +'_mean_'+ key) for key in  M_AS_CF_list]
    [M_AS_CF_agg_list.append(title_name +'_mode_'+ key) for key in  M_AS_CF_list]
    [M_AS_CF_agg_list.append(title_name +'_std_'+ key) for key in  M_AS_CF_list]
    
    return M_AS_CF_agg_list

def generate_C_AS_CB_features():
    
    title_name = 'C_AS_CB' 
    C_AS_CB_list = ["pick_crystal", "place_crystal", "air_release", "withdraw_crystal", \
                    "win","miss", "per_win", "count_result", "acc", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    C_AS_CB_agg_list = []
    [C_AS_CB_agg_list.append(title_name +'_mean_'+ key) for key in  C_AS_CB_list]
    [C_AS_CB_agg_list.append(title_name +'_mode_'+ key) for key in  C_AS_CB_list]
    [C_AS_CB_agg_list.append(title_name +'_std_'+ key) for key in  C_AS_CB_list]
    
    return C_AS_CB_agg_list

def generate_C_AS_CS_features():
    
    title_name = 'C_AS_CS' 
    C_AS_CS_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "win", "miss", "per_win", "count_result", "acc",\
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    C_AS_CS_agg_list = []
    [C_AS_CS_agg_list.append(title_name +'_mean_'+ key) for key in  C_AS_CS_list]
    [C_AS_CS_agg_list.append(title_name +'_mode_'+ key) for key in  C_AS_CS_list]
    [C_AS_CS_agg_list.append(title_name +'_std_'+ key) for key in  C_AS_CS_list]
    
    return C_AS_CS_agg_list

# Initiate feature list for all the activity

def generate_M_AC_SB_features():
    
    title_name = 'M_AC_SB' 
    M_AC_SB_list = ["shovel_placement", "castle_placement", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    M_AC_SB_agg_list = []
    [M_AC_SB_agg_list.append(title_name +'_mean_'+ key) for key in  M_AC_SB_list]
    [M_AC_SB_agg_list.append(title_name +'_mode_'+ key) for key in  M_AC_SB_list]
    [M_AC_SB_agg_list.append(title_name +'_std_'+ key) for key in  M_AC_SB_list]
    
    return M_AC_SB_agg_list

def generate_M_AC_WH_features():
    
    title_name = 'M_AC_WH' 
    M_AC_WH_list = ["rain_cloud", "pool_filled", "fill_cloud", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    M_AC_WH_agg_list = []
    [M_AC_WH_agg_list.append(title_name +'_mean_'+ key) for key in  M_AC_WH_list]
    [M_AC_WH_agg_list.append(title_name +'_mode_'+ key) for key in  M_AC_WH_list]
    [M_AC_WH_agg_list.append(title_name +'_std_'+ key) for key in  M_AC_WH_list]
    
    return M_AC_WH_agg_list

def generate_M_AC_BF_features():
    
    title_name = 'M_AC_BF' 
    M_AC_BF_list = ["bottles_filled", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    M_AC_BF_agg_list = []
    [M_AC_BF_agg_list.append(title_name +'_mean_'+ key) for key in  M_AC_BF_list]
    [M_AC_BF_agg_list.append(title_name +'_mode_'+ key) for key in  M_AC_BF_list]
    [M_AC_BF_agg_list.append(title_name +'_std_'+ key) for key in  M_AC_BF_list]
    
    return M_AC_BF_agg_list

def generate_T_AC_BM_features():
    
    title_name = 'T_AC_BM' 
    T_AC_BM_list = ["bugs_measured", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    T_AC_BM_agg_list = []
    [T_AC_BM_agg_list.append(title_name +'_mean_'+ key) for key in  T_AC_BM_list]
    [T_AC_BM_agg_list.append(title_name +'_mode_'+ key) for key in  T_AC_BM_list]
    [T_AC_BM_agg_list.append(title_name +'_std_'+ key) for key in  T_AC_BM_list]
    
    return T_AC_BM_agg_list

def generate_T_AC_FW_features():
    
    title_name = 'T_AC_FW' 
    T_AC_FW_list = ["flowers_picked", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    T_AC_FW_agg_list = []
    [T_AC_FW_agg_list.append(title_name +'_mean_'+ key) for key in  T_AC_FW_list]
    [T_AC_FW_agg_list.append(title_name +'_mode_'+ key) for key in  T_AC_FW_list]
    [T_AC_FW_agg_list.append(title_name +'_std_'+ key) for key in  T_AC_FW_list]
    
    return T_AC_FW_agg_list

def generate_T_AC_F_features():
    
    title_name = 'T_AC_F' 
    T_AC_F_list = ["drag_rockets_count", "launch", "fail","per_launch", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    T_AC_F_agg_list = []
    [T_AC_F_agg_list.append(title_name +'_mean_'+ key) for key in  T_AC_F_list]
    [T_AC_F_agg_list.append(title_name +'_mode_'+ key) for key in  T_AC_F_list]
    [T_AC_F_agg_list.append(title_name +'_std_'+ key) for key in  T_AC_F_list]
    
    return T_AC_F_agg_list

def generate_C_AC_CB_features():
    
    title_name = 'C_AC_CB' 
    C_AC_CB_list = ["chicken_placement", "pig_placement", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    C_AC_CB_agg_list = []
    [C_AC_CB_agg_list.append(title_name +'_mean_'+ key) for key in  C_AC_CB_list]
    [C_AC_CB_agg_list.append(title_name +'_mode_'+ key) for key in  C_AC_CB_list]
    [C_AC_CB_agg_list.append(title_name +'_std_'+ key) for key in  C_AC_CB_list]
    
    return C_AC_CB_agg_list

def generate_C_AC_ED_features():
    
    title_name = 'C_AC_ED' 
    C_AC_ED_list = ["click_bird", "click_rock", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    C_AC_ED_agg_list = []
    [C_AC_ED_agg_list.append(title_name +'_mean_'+ key) for key in  C_AC_ED_list]
    [C_AC_ED_agg_list.append(title_name +'_mode_'+ key) for key in  C_AC_ED_list]
    [C_AC_ED_agg_list.append(title_name +'_std_'+ key) for key in  C_AC_ED_list]
    
    return C_AC_ED_agg_list

# Initiate feature list for all the games

def generate_M_G_SAD_features():
    
    title_name = 'M_G_SAD' 
    M_G_SAD_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "levels_completed", "clicked_pic",\
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    M_G_SAD_agg_list = []
    [M_G_SAD_agg_list.append(title_name +'_mean_'+ key) for key in  M_G_SAD_list]
    [M_G_SAD_agg_list.append(title_name +'_mode_'+ key) for key in  M_G_SAD_list]
    [M_G_SAD_agg_list.append(title_name +'_std_'+ key) for key in  M_G_SAD_list]
    
    return M_G_SAD_agg_list

def generate_M_G_DDR_features():
    
    title_name = 'M_G_DDR' 
    M_G_DDR_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "skip_tutorial", "rounds",\
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    M_G_DDR_agg_list = []
    [M_G_DDR_agg_list.append(title_name +'_mean_'+ key) for key in  M_G_DDR_list]
    [M_G_DDR_agg_list.append(title_name +'_mode_'+ key) for key in  M_G_DDR_list]
    [M_G_DDR_agg_list.append(title_name +'_std_'+ key) for key in  M_G_DDR_list]
    
    return M_G_DDR_agg_list

def generate_M_G_BB_features():
    
    title_name = 'M_G_BB' 
    M_G_BB_list = ["correct_guess", "incorrect_guess", "per_correct_guess", "count_guess", \
                   "interactive_clicks", "drag_toy", \
                   "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    M_G_BB_agg_list = []
    [M_G_BB_agg_list.append(title_name +'_mean_'+ key) for key in  M_G_BB_list]
    [M_G_BB_agg_list.append(title_name +'_mode_'+ key) for key in  M_G_BB_list]
    [M_G_BB_agg_list.append(title_name +'_std_'+ key) for key in  M_G_BB_list]
    
    return M_G_BB_agg_list

def generate_M_G_DD_features():
    
    title_name = 'M_G_DD' 
    M_G_DD_list = ["correct_tap", "incorrect_tap", "per_correct_tap", "count_tap", "rounds_played",  \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    M_G_DD_agg_list = []
    [M_G_DD_agg_list.append(title_name +'_mean_'+ key) for key in  M_G_DD_list]
    [M_G_DD_agg_list.append(title_name +'_mode_'+ key) for key in  M_G_DD_list]
    [M_G_DD_agg_list.append(title_name +'_std_'+ key) for key in  M_G_DD_list]
    
    return M_G_DD_agg_list

def generate_T_G_ASS_features():
    
    title_name = 'T_G_ASS' 
    T_G_ASS_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "rounds_played", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    T_G_ASS_agg_list = []
    [T_G_ASS_agg_list.append(title_name +'_mean_'+ key) for key in  T_G_ASS_list]
    [T_G_ASS_agg_list.append(title_name +'_mode_'+ key) for key in  T_G_ASS_list]
    [T_G_ASS_agg_list.append(title_name +'_std_'+ key) for key in  T_G_ASS_list]
    
    return T_G_ASS_agg_list

def generate_T_G_AS_features():
    
    title_name = 'T_G_AS' 
    T_G_AS_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                   "win", "miss", "per_win", "count_result", "skip_tutorial", "tutorials_watched", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    T_G_AS_agg_list = []
    [T_G_AS_agg_list.append(title_name +'_mean_'+ key) for key in  T_G_AS_list]
    [T_G_AS_agg_list.append(title_name +'_mode_'+ key) for key in  T_G_AS_list]
    [T_G_AS_agg_list.append(title_name +'_std_'+ key) for key in  T_G_AS_list]
    
    return T_G_AS_agg_list

def generate_T_G_CR_features():
    
    title_name = 'T_G_CR' 
    T_G_CR_list = ["correct_count", "incorrect_count", "per_correct_count", "count_count", "drag_object", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    T_G_CR_agg_list = []
    [T_G_CR_agg_list.append(title_name +'_mean_'+ key) for key in  T_G_CR_list]
    [T_G_CR_agg_list.append(title_name +'_mode_'+ key) for key in  T_G_CR_list]
    [T_G_CR_agg_list.append(title_name +'_std_'+ key) for key in  T_G_CR_list]
    
    return T_G_CR_agg_list

def generate_C_G_CT_features():
    
    title_name = 'C_G_CT' 
    C_G_CT_list = ["correct_placement", "incorrect_pacement", "per_correct_placement", "count_placement", "rounds", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    C_G_CT_agg_list = []
    [C_G_CT_agg_list.append(title_name +'_mean_'+ key) for key in  C_G_CT_list]
    [C_G_CT_agg_list.append(title_name +'_mode_'+ key) for key in  C_G_CT_list]
    [C_G_CT_agg_list.append(title_name +'_std_'+ key) for key in  C_G_CT_list]
    
    return C_G_CT_agg_list

def generate_C_G_HC_features():
    
    title_name = 'C_G_HC' 
    C_G_HC_list = ["win", "miss", "per_win", "count_result", "rounds", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    C_G_HC_agg_list = []
    [C_G_HC_agg_list.append(title_name +'_mean_'+ key) for key in  C_G_HC_list]
    [C_G_HC_agg_list.append(title_name +'_mode_'+ key) for key in  C_G_HC_list]
    [C_G_HC_agg_list.append(title_name +'_std_'+ key) for key in  C_G_HC_list]
    
    return C_G_HC_agg_list

def generate_C_G_LL_features():
    
    title_name = 'C_G_LL' 
    C_G_LL_list = ["add", "remove", "rounds", "per_add", "count_placement", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    C_G_LL_agg_list = []
    [C_G_LL_agg_list.append(title_name +'_mean_'+ key) for key in  C_G_LL_list]
    [C_G_LL_agg_list.append(title_name +'_mode_'+ key) for key in  C_G_LL_list]
    [C_G_LL_agg_list.append(title_name +'_std_'+ key) for key in  C_G_LL_list]
    
    return C_G_LL_agg_list

def generate_C_G_PB_features():
    
    title_name = 'C_G_PB' 
    C_G_PB_list = ["correct_balance", "incorrect_balance", "per_correct_balance", "count_balance", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    
    C_G_PB_agg_list = []
    [C_G_PB_agg_list.append(title_name +'_mean_'+ key) for key in  C_G_PB_list]
    [C_G_PB_agg_list.append(title_name +'_mode_'+ key) for key in  C_G_PB_list]
    [C_G_PB_agg_list.append(title_name +'_std_'+ key) for key in  C_G_PB_list]
    
    return C_G_PB_agg_list

# Initiate count feature list for all the titles

def generate_count_features_raw_dict():
    # Extract all the titles present in data
    titles = set(train.title.unique())
    # create count feature name for all the titles
    count_features_list = []
    [count_features_list.append(title+'_count') for title in  titles]

    return count_features_list

In [ ]:
def generate_final_raw_features_dict():
    
    # Initiate list for assessment
    T_AS_MS_agg_list = generate_T_AS_MS_features()
    T_AS_BM_agg_list = generate_T_AS_BM_features()
    M_AS_CF_agg_list = generate_M_AS_CF_features()
    C_AS_CB_agg_list = generate_C_AS_CB_features()
    C_AS_CS_agg_list = generate_C_AS_CS_features()
    # Initiate list for activity
    M_AC_SB_agg_list = generate_M_AC_SB_features()
    M_AC_WH_agg_list = generate_M_AC_WH_features()
    M_AC_BF_agg_list = generate_M_AC_BF_features()
    T_AC_BM_agg_list = generate_T_AC_BM_features()
    T_AC_FW_agg_list = generate_T_AC_FW_features()
    T_AC_F_agg_list = generate_T_AC_F_features()
    C_AC_CB_agg_list = generate_C_AC_CB_features()
    C_AC_ED_agg_list = generate_C_AC_ED_features()
    # Initiate list for games
    M_G_SAD_agg_list = generate_M_G_SAD_features()
    M_G_DDR_agg_list = generate_M_G_DDR_features()
    M_G_BB_agg_list = generate_M_G_BB_features()
    M_G_DD_agg_list = generate_M_G_DD_features()
    T_G_ASS_agg_list = generate_T_G_ASS_features()
    T_G_AS_agg_list = generate_T_G_AS_features()
    T_G_CR_agg_list = generate_T_G_CR_features()
    C_G_CT_agg_list = generate_C_G_CT_features()
    C_G_PB_agg_list = generate_C_G_PB_features()
    C_G_HC_agg_list = generate_C_G_HC_features()
    C_G_LL_agg_list = generate_C_G_LL_features()
    
    count_features_list = generate_count_features_raw_dict()
    
    # Initiate dictionary for assessment
    T_AS_MS_raw_dict = dict(zip(T_AS_MS_agg_list, np.ones(len(T_AS_MS_agg_list), dtype='int8')*-1))
    T_AS_BM_raw_dict = dict(zip(T_AS_BM_agg_list, np.ones(len(T_AS_BM_agg_list), dtype='int8')*-1))
    M_AS_CF_raw_dict = dict(zip(M_AS_CF_agg_list, np.ones(len(M_AS_CF_agg_list), dtype='int8')*-1))
    C_AS_CB_raw_dict = dict(zip(C_AS_CB_agg_list, np.ones(len(C_AS_CB_agg_list), dtype='int8')*-1))
    C_AS_CS_raw_dict = dict(zip(C_AS_CS_agg_list, np.ones(len(C_AS_CS_agg_list), dtype='int8')*-1))
    # Initiate dictionary for activity
    M_AC_SB_raw_dict = dict(zip(M_AC_SB_agg_list, np.ones(len(M_AC_SB_agg_list), dtype='int8')*-1))
    M_AC_WH_raw_dict = dict(zip(M_AC_WH_agg_list, np.ones(len(M_AC_WH_agg_list), dtype='int8')*-1))
    M_AC_BF_raw_dict = dict(zip(M_AC_BF_agg_list, np.ones(len(M_AC_BF_agg_list), dtype='int8')*-1))
    T_AC_BM_raw_dict = dict(zip(T_AC_BM_agg_list, np.ones(len(T_AC_BM_agg_list), dtype='int8')*-1))
    T_AC_FW_raw_dict = dict(zip(T_AC_FW_agg_list, np.ones(len(T_AC_FW_agg_list), dtype='int8')*-1))
    T_AC_F_raw_dict = dict(zip(T_AC_F_agg_list, np.ones(len(T_AC_F_agg_list), dtype='int8')*-1))
    C_AC_CB_raw_dict = dict(zip(C_AC_CB_agg_list, np.ones(len(C_AC_CB_agg_list), dtype='int8')*-1))
    C_AC_ED_raw_dict = dict(zip(C_AC_ED_agg_list, np.ones(len(C_AC_ED_agg_list), dtype='int8')*-1))
    # Initiate dictionary for games
    M_G_SAD_raw_dict = dict(zip(M_G_SAD_agg_list, np.ones(len(M_G_SAD_agg_list), dtype='int8')*-1))
    M_G_DDR_raw_dict = dict(zip(M_G_DDR_agg_list, np.ones(len(M_G_DDR_agg_list), dtype='int8')*-1))
    M_G_BB_raw_dict = dict(zip(M_G_BB_agg_list, np.ones(len(M_G_BB_agg_list), dtype='int8')*-1))
    M_G_DD_raw_dict = dict(zip(M_G_DD_agg_list, np.ones(len(M_G_DD_agg_list), dtype='int8')*-1))
    T_G_ASS_raw_dict = dict(zip(T_G_ASS_agg_list, np.ones(len(T_G_ASS_agg_list), dtype='int8')*-1))
    T_G_AS_raw_dict = dict(zip(T_G_AS_agg_list, np.ones(len(T_G_AS_agg_list), dtype='int8')*-1))
    T_G_CR_raw_dict = dict(zip(T_G_CR_agg_list, np.ones(len(T_G_CR_agg_list), dtype='int8')*-1))
    C_G_CT_raw_dict = dict(zip(C_G_CT_agg_list, np.ones(len(C_G_CT_agg_list), dtype='int8')*-1))
    C_G_PB_raw_dict = dict(zip(C_G_PB_agg_list, np.ones(len(C_G_PB_agg_list), dtype='int8')*-1))
    C_G_HC_raw_dict = dict(zip(C_G_HC_agg_list, np.ones(len(C_G_HC_agg_list), dtype='int8')*-1))
    C_G_LL_raw_dict = dict(zip(C_G_LL_agg_list, np.ones(len(C_G_LL_agg_list), dtype='int8')*-1))
    
    count_features_raw_dict = dict(zip(count_features_list, np.zeros(len(count_features_list), dtype='int8')))
    
    raw_features_dict = {}
    # add assessment features to final dictionary 
    raw_features_dict.update(T_AS_MS_raw_dict)
    raw_features_dict.update(T_AS_BM_raw_dict)
    raw_features_dict.update(M_AS_CF_raw_dict)
    raw_features_dict.update(C_AS_CB_raw_dict)
    raw_features_dict.update(C_AS_CS_raw_dict)
    # add activity features to final dictionary
    raw_features_dict.update(M_AC_SB_raw_dict)
    raw_features_dict.update(M_AC_WH_raw_dict)
    raw_features_dict.update(M_AC_BF_raw_dict)
    raw_features_dict.update(T_AC_BM_raw_dict)
    raw_features_dict.update(T_AC_FW_raw_dict)
    raw_features_dict.update(T_AC_F_raw_dict)
    raw_features_dict.update(C_AC_CB_raw_dict)
    raw_features_dict.update(C_AC_ED_raw_dict)
    # add games features to final dictionary
    raw_features_dict.update(M_G_SAD_raw_dict)
    raw_features_dict.update(M_G_DDR_raw_dict)
    raw_features_dict.update(M_G_BB_raw_dict)
    raw_features_dict.update(M_G_DD_raw_dict)
    raw_features_dict.update(T_G_ASS_raw_dict)
    raw_features_dict.update(T_G_AS_raw_dict)
    raw_features_dict.update(T_G_CR_raw_dict)
    raw_features_dict.update(C_G_CT_raw_dict)
    raw_features_dict.update(C_G_PB_raw_dict)
    raw_features_dict.update(C_G_HC_raw_dict)
    raw_features_dict.update(C_G_LL_raw_dict)
    
    raw_features_dict.update(count_features_raw_dict)
    
    return raw_features_dict


In [ ]:
# titles features extraction 
# Assessment features

def get_T_AS_MS_features(title_session):
    
    title_name = title_session.iloc[0].title
    
    T_AS_MS_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "win", "miss", "per_win", "count_result", "acc", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed"
                   ]
    T_AS_MS_dict = dict(zip(T_AS_MS_list, [[] for i in range(len(T_AS_MS_list))]))

    for game, game_session in title_session.groupby('game_session', sort=False):
        
        placement = game_session[(game_session['event_code']==4020)].correct_flag.values
        correct_placement = placement.sum()
        incorrect_placement = len(placement) - correct_placement 
        
        per_correct_placement  = correct_placement/ (len(placement) + 0.01)
        count_placement = len(placement)
        
        result = game_session[(game_session['event_code']==4100)].correct_flag.values
        win = result.sum()
        miss = len(result) - win
        
        per_win = win/(len(result) + 0.01)
        count_result = len(result)
        
        acc = np.where((win == 0), 0, np.where((miss == 0), 3, np.where((miss == 1),  2, 1)))
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        T_AS_MS_dict['correct_placement'].append(correct_placement)
        T_AS_MS_dict['incorrect_placement'].append(incorrect_placement)
        T_AS_MS_dict['per_correct_placement'].append(per_correct_placement)
        T_AS_MS_dict['count_placement'].append(count_placement)
        T_AS_MS_dict['win'].append(win)
        T_AS_MS_dict['miss'].append(miss)
        T_AS_MS_dict['per_win'].append(per_win)
        T_AS_MS_dict['count_result'].append(count_result)
        
        T_AS_MS_dict['acc'].append(acc)
        
        T_AS_MS_dict['extra_clicks'].append(extra_clicks)
        T_AS_MS_dict['help_count'].append(help_count)
        T_AS_MS_dict['game_time'].append(game_time)
        T_AS_MS_dict['event_count'].append(event_count)
        T_AS_MS_dict['speed'].append(speed)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    
    # get final aggregate features
    T_AS_MS_agg_dict = {}
    [T_AS_MS_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in T_AS_MS_dict.items()]
    
    return T_AS_MS_agg_dict

def get_T_AS_BM_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    T_AS_BM_list = ["correct_drag", "incorrect_drag", "per_correct_drag", "count_drag", \
                    "drag_win", "drag_miss", "per_drag_win", "count_drag_result", \
                    "correct_hat", "incorrect_hat", "per_correct_hat", "count_hat", \
                    "hat_win", "hat_miss", "per_hat_win", "count_hat_result", \
                    "acc", "extra_clicks", "help_count", "game_time", "event_count","speed" \
                   ]
    T_AS_BM_dict = dict(zip(T_AS_BM_list, [[] for i in range(len(T_AS_BM_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
                
        drag = game_session[(game_session['event_code']==4025)].correct_flag.values
        correct_drag = drag.sum()
        incorrect_drag = len(drag)-correct_drag
        
        per_correct_drag = correct_drag/(len(drag) + 0.01)
        count_drag = len(drag)
        
        drag_result = game_session[(game_session['event_code']==4110)].correct_flag.values
        drag_win = drag_result.sum()
        drag_miss = len(drag_result) - drag_win
        
        per_drag_win = drag_win/(len(drag_result) + 0.01)
        count_drag_result = len(drag_result)
        
        acc = np.where((drag_win == 0), 0, np.where((drag_miss == 0), 3, np.where((drag_miss == 1),  2, 1)))
        
        hat = game_session[(game_session['event_code']==4020)].correct_flag.values
        correct_hat = hat.sum()
        incorrect_hat = len(hat)-correct_hat
        
        per_correct_hat = correct_hat/(len(hat) + 0.01)
        count_hat = len(hat)
        
        hat_result = game_session[(game_session['event_code']==4100)].correct_flag.values       
        hat_win = hat_result.sum()
        hat_miss = len(hat_result) - hat_win   
        
        per_hat_win = hat_win/(len(hat_result) + 0.01)
        count_hat_result = len(hat_result)
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        T_AS_BM_dict['correct_drag'].append(correct_drag)
        T_AS_BM_dict['incorrect_drag'].append(incorrect_drag)
        T_AS_BM_dict['per_correct_drag'].append(per_correct_drag)
        T_AS_BM_dict['count_drag'].append(count_drag)
        T_AS_BM_dict['drag_win'].append(drag_win)
        T_AS_BM_dict['drag_miss'].append(drag_miss)
        T_AS_BM_dict['per_drag_win'].append(per_drag_win)
        T_AS_BM_dict['count_drag_result'].append(count_drag_result)
        T_AS_BM_dict['correct_hat'].append(correct_hat)
        T_AS_BM_dict['incorrect_hat'].append(incorrect_hat)
        T_AS_BM_dict['per_correct_hat'].append(per_correct_hat)
        T_AS_BM_dict['count_hat'].append(count_hat)
        T_AS_BM_dict['hat_win'].append(hat_win)
        T_AS_BM_dict['hat_miss'].append(hat_miss)
        T_AS_BM_dict['per_hat_win'].append(per_hat_win)
        T_AS_BM_dict['count_hat_result'].append(count_hat_result)
        T_AS_BM_dict['acc'].append(acc)
        
        T_AS_BM_dict['extra_clicks'].append(extra_clicks)
        T_AS_BM_dict['help_count'].append(help_count)
        T_AS_BM_dict['game_time'].append(game_time)
        T_AS_BM_dict['event_count'].append(event_count)
        T_AS_BM_dict['speed'].append(speed)
    
    # get final aggregate features
    T_AS_BM_agg_dict = {}
    [T_AS_BM_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in T_AS_BM_dict.items()]
    
    return T_AS_BM_agg_dict

def get_M_AS_CF_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    M_AS_CF_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "win", "miss", "per_win", "count_result", "acc", \
                    "correct_bucket","incorrect_bucket", "per_bucket_click", "count_bucket_click", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    M_AS_CF_dict = dict(zip(M_AS_CF_list, [[] for i in range(len(M_AS_CF_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        placement = game_session[(game_session['event_code']==4020)].correct_flag.values
        correct_placement = placement.sum()
        incorrect_placement = len(placement)-correct_placement
        
        per_correct_placement  = correct_placement/ (len(placement) + 0.01)
        count_placement = len(placement)
        
        result = game_session[(game_session['event_code']==4100)].correct_flag.values
        win = result.sum()
        miss = len(result)-win
        
        per_win = win/(len(result) + 0.01)
        count_result = len(result)
        
        acc = np.where((win == 0), 0, np.where((miss == 0), 3, np.where((miss == 1),  2, 1)))
        
        bucket_click = game_session[(game_session['event_code']==4025)].correct_flag.values
        correct_bucket = bucket_click.sum()
        incorrect_bucket = len(bucket_click)-correct_bucket       
        
        per_bucket_click = correct_bucket/(len(bucket_click) + 0.01)
        count_bucket_click = len(bucket_click)
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        M_AS_CF_dict['correct_placement'].append(correct_placement)
        M_AS_CF_dict['incorrect_placement'].append(incorrect_placement)
        M_AS_CF_dict['per_correct_placement'].append(per_correct_placement)
        M_AS_CF_dict['count_placement'].append(count_placement)
        M_AS_CF_dict['win'].append(win)
        M_AS_CF_dict['miss'].append(miss)
        M_AS_CF_dict['per_win'].append(per_win)
        M_AS_CF_dict['count_result'].append(count_result)
        M_AS_CF_dict['correct_bucket'].append(correct_bucket)
        M_AS_CF_dict['incorrect_bucket'].append(incorrect_bucket)
        M_AS_CF_dict['per_bucket_click'].append(per_bucket_click)
        M_AS_CF_dict['count_bucket_click'].append(count_bucket_click)
        M_AS_CF_dict['acc'].append(acc)  
        
        M_AS_CF_dict['extra_clicks'].append(extra_clicks)
        M_AS_CF_dict['help_count'].append(help_count)
        M_AS_CF_dict['game_time'].append(game_time)
        M_AS_CF_dict['event_count'].append(event_count)
        M_AS_CF_dict['speed'].append(speed)
    
    # get final aggregate features
    M_AS_CF_agg_dict = {}
    [M_AS_CF_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in M_AS_CF_dict.items()]
    
    return M_AS_CF_agg_dict

def get_C_AS_CB_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    C_AS_CB_list = ["pick_crystal", "place_crystal", "air_release", "withdraw_crystal", \
                    "win","miss", "per_win", "count_result", "acc", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    C_AS_CB_dict = dict(zip(C_AS_CB_list, [[] for i in range(len(C_AS_CB_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        pick_crystal = (game_session.event_code==4030).sum()
        place_crystal = (game_session.event_code==4020).sum()
        air_release = (game_session.event_code==4035).sum()
        withdraw_crystal = (game_session.event_code==4040).sum()      
        
        result = game_session[(game_session['event_code']==4100)].correct_flag.values
        win = result.sum()
        miss = len(result)-win
        
        per_win = win/(len(result) + 0.01)
        count_result = len(result)
        
        acc = np.where((win == 0), 0, np.where((miss == 0), 3, np.where((miss == 1),  2, 1)))
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        C_AS_CB_dict['pick_crystal'].append(pick_crystal)
        C_AS_CB_dict['place_crystal'].append(place_crystal)
        C_AS_CB_dict['air_release'].append(air_release)
        C_AS_CB_dict['withdraw_crystal'].append(withdraw_crystal)  
        C_AS_CB_dict['win'].append(win)
        C_AS_CB_dict['miss'].append(miss)
        C_AS_CB_dict['per_win'].append(per_win)
        C_AS_CB_dict['count_result'].append(count_result)
        C_AS_CB_dict['acc'].append(acc)
        
        C_AS_CB_dict['extra_clicks'].append(extra_clicks)
        C_AS_CB_dict['help_count'].append(help_count)
        C_AS_CB_dict['game_time'].append(game_time)
        C_AS_CB_dict['event_count'].append(event_count)
        C_AS_CB_dict['speed'].append(speed)
    
    # get final aggregate features
    C_AS_CB_agg_dict = {}
    [C_AS_CB_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in C_AS_CB_dict.items()]
    
    return C_AS_CB_agg_dict

def get_C_AS_CS_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    C_AS_CS_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "win", "miss", "per_win", "count_result", "acc",\
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    C_AS_CS_dict = dict(zip(C_AS_CS_list, [[] for i in range(len(C_AS_CS_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        placement = game_session[(game_session['event_code']==4020)].correct_flag.values
        correct_placement = placement.sum()
        incorrect_placement = len(placement)-correct_placement
        
        per_correct_placement  = correct_placement/ (len(placement) + 0.01)
        count_placement = len(placement)
        
        result = game_session[(game_session['event_code']==4100)].correct_flag.values
        win = result.sum()
        miss = len(result)-win      
        
        per_win = win/(len(result) + 0.01)
        count_result = len(result)
        
        acc = np.where((win == 0), 0, np.where((miss == 0), 3, np.where((miss == 1),  2, 1)))
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        C_AS_CS_dict['correct_placement'].append(correct_placement)
        C_AS_CS_dict['incorrect_placement'].append(incorrect_placement)
        C_AS_CS_dict['per_correct_placement'].append(per_correct_placement)
        C_AS_CS_dict['count_placement'].append(count_placement)
        C_AS_CS_dict['win'].append(win)
        C_AS_CS_dict['miss'].append(miss)
        C_AS_CS_dict['per_win'].append(per_win)
        C_AS_CS_dict['count_result'].append(count_result)
        C_AS_CS_dict['acc'].append(acc)
        
        C_AS_CS_dict['extra_clicks'].append(extra_clicks)
        C_AS_CS_dict['help_count'].append(help_count)
        C_AS_CS_dict['game_time'].append(game_time)
        C_AS_CS_dict['event_count'].append(event_count)
        C_AS_CS_dict['speed'].append(speed)
    
    # get final aggregate features
    C_AS_CS_agg_dict = {}
    [C_AS_CS_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in C_AS_CS_dict.items()]
    
    return C_AS_CS_agg_dict

# Activity features

def get_M_AC_SB_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    M_AC_SB_list = ["shovel_placement", "castle_placement", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    M_AC_SB_dict = dict(zip(M_AC_SB_list, [[] for i in range(len(M_AC_SB_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        shovel_placement = (game_session.event_code==4021).sum()
        castle_placement = (game_session.event_code==4020).sum()
                
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        M_AC_SB_dict['shovel_placement'].append(shovel_placement)
        M_AC_SB_dict['castle_placement'].append(castle_placement)
        
        M_AC_SB_dict['extra_clicks'].append(extra_clicks)
        M_AC_SB_dict['help_count'].append(help_count)
        M_AC_SB_dict['game_time'].append(game_time)
        M_AC_SB_dict['event_count'].append(event_count)
        M_AC_SB_dict['speed'].append(speed)
    
    # get final aggregate features
    M_AC_SB_agg_dict = {}
    [M_AC_SB_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in M_AC_SB_dict.items()]
    
    return M_AC_SB_agg_dict

def get_M_AC_WH_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    M_AC_WH_list = ["rain_cloud", "pool_filled", "fill_cloud", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    M_AC_WH_dict = dict(zip(M_AC_WH_list, [[] for i in range(len(M_AC_WH_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        rain_cloud = (game_session['event_code']==4025).sum()
        pool_filled = (game_session['event_code']==5000).sum()
        fill_cloud = (game_session['event_code']==4020).sum()
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        M_AC_WH_dict['rain_cloud'].append(rain_cloud)
        M_AC_WH_dict['pool_filled'].append(pool_filled)
        M_AC_WH_dict['fill_cloud'].append(fill_cloud)
        
        M_AC_WH_dict['extra_clicks'].append(extra_clicks)
        M_AC_WH_dict['help_count'].append(help_count)
        M_AC_WH_dict['game_time'].append(game_time)
        M_AC_WH_dict['event_count'].append(event_count)
        M_AC_WH_dict['speed'].append(speed)
    
    # get final aggregate features
    M_AC_WH_agg_dict = {}
    [M_AC_WH_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in M_AC_WH_dict.items()]
    
    return M_AC_WH_agg_dict

def get_M_AC_BF_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    M_AC_BF_list = ["bottles_filled", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    M_AC_BF_dict = dict(zip(M_AC_BF_list, [[] for i in range(len(M_AC_BF_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        bottles_filled = (game_session['event_code']==2030).sum()
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        M_AC_BF_dict['bottles_filled'].append(bottles_filled)
        
        M_AC_BF_dict['extra_clicks'].append(extra_clicks)
        M_AC_BF_dict['help_count'].append(help_count)
        M_AC_BF_dict['game_time'].append(game_time)
        M_AC_BF_dict['event_count'].append(event_count)
        M_AC_BF_dict['speed'].append(speed)
    
    # get final aggregate features
    M_AC_BF_agg_dict = {}
    [M_AC_BF_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in M_AC_BF_dict.items()]
    
    return M_AC_BF_agg_dict

def get_T_AC_BM_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    T_AC_BM_list = ["bugs_measured", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    T_AC_BM_dict = dict(zip(T_AC_BM_list, [[] for i in range(len(T_AC_BM_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
        
        bugs_measured = (game_session['event_code']==4025).sum()

        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        T_AC_BM_dict['bugs_measured'].append(bugs_measured)
        
        T_AC_BM_dict['extra_clicks'].append(extra_clicks)
        T_AC_BM_dict['help_count'].append(help_count)
        T_AC_BM_dict['game_time'].append(game_time)
        T_AC_BM_dict['event_count'].append(event_count)
        T_AC_BM_dict['speed'].append(speed)
    
    # get final aggregate features
    T_AC_BM_agg_dict = {}
    [T_AC_BM_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in T_AC_BM_dict.items()]
    
    return T_AC_BM_agg_dict

def get_T_AC_FW_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    T_AC_FW_list = ["flowers_picked", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    T_AC_FW_dict = dict(zip(T_AC_FW_list, [[] for i in range(len(T_AC_FW_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        flowers_picked = (game_session['event_code']==4022).sum()
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        T_AC_FW_dict['flowers_picked'].append(flowers_picked)
        
        T_AC_FW_dict['extra_clicks'].append(extra_clicks)
        T_AC_FW_dict['help_count'].append(help_count)
        T_AC_FW_dict['game_time'].append(game_time)
        T_AC_FW_dict['event_count'].append(event_count)
        T_AC_FW_dict['speed'].append(speed)
    
    # get final aggregate features
    T_AC_FW_agg_dict = {}
    [T_AC_FW_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in T_AC_FW_dict.items()]
    
    return T_AC_FW_agg_dict

def get_T_AC_F_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    T_AC_F_list = ["drag_rockets_count", "launch", "fail","per_launch", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    T_AC_F_dict = dict(zip(T_AC_F_list, [[] for i in range(len(T_AC_F_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        drag_rockets = game_session[(game_session['event_code']==4020)].correct_flag.values
        drag_rockets_count = len(drag_rockets)
        launch = drag_rockets.sum()
        fail = drag_rockets_count-launch 
        
        per_launch = launch/(len(drag_rockets) + 0.01)
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        T_AC_F_dict['drag_rockets_count'].append(drag_rockets_count)
        T_AC_F_dict['launch'].append(launch)
        T_AC_F_dict['fail'].append(fail)
        T_AC_F_dict['per_launch'].append(per_launch)
        
        T_AC_F_dict['extra_clicks'].append(extra_clicks)
        T_AC_F_dict['help_count'].append(help_count)
        T_AC_F_dict['game_time'].append(game_time)
        T_AC_F_dict['event_count'].append(event_count)
        T_AC_F_dict['speed'].append(speed)
        
    
    # get final aggregate features
    T_AC_F_agg_dict = {}
    [T_AC_F_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in T_AC_F_dict.items()]
    
    return T_AC_F_agg_dict

def get_C_AC_CB_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    C_AC_CB_list = ["chicken_placement", "pig_placement", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    C_AC_CB_dict = dict(zip(C_AC_CB_list, [[] for i in range(len(C_AC_CB_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        chicken_placement = (game_session['event_code']==4020).sum()
        pig_placement = (game_session['event_code']==4022).sum()
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        C_AC_CB_dict['chicken_placement'].append(chicken_placement)
        C_AC_CB_dict['pig_placement'].append(pig_placement)

        C_AC_CB_dict['extra_clicks'].append(extra_clicks)
        C_AC_CB_dict['help_count'].append(help_count)
        C_AC_CB_dict['game_time'].append(game_time)
        C_AC_CB_dict['event_count'].append(event_count)
        C_AC_CB_dict['speed'].append(speed)
    
    # get final aggregate features
    C_AC_CB_agg_dict = {}
    [C_AC_CB_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in C_AC_CB_dict.items()]
    
    return C_AC_CB_agg_dict

def get_C_AC_ED_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    C_AC_ED_list = ["click_bird", "click_rock", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    C_AC_ED_dict = dict(zip(C_AC_ED_list, [[] for i in range(len(C_AC_ED_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        click_bird = (game_session['event_code']==4025).sum()
        click_rock = (game_session['event_code']==4020).sum()
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        C_AC_ED_dict['click_bird'].append(click_bird)
        C_AC_ED_dict['click_rock'].append(click_rock)
        
        C_AC_ED_dict['extra_clicks'].append(extra_clicks)
        C_AC_ED_dict['help_count'].append(help_count)
        C_AC_ED_dict['game_time'].append(game_time)
        C_AC_ED_dict['event_count'].append(event_count)
        C_AC_ED_dict['speed'].append(speed)
    
    # get final aggregate features
    C_AC_ED_agg_dict = {}
    [C_AC_ED_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in C_AC_ED_dict.items()]
    
    return C_AC_ED_agg_dict


# Games Features

def get_M_G_SAD_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    M_G_SAD_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "levels_completed", "clicked_pic",\
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    M_G_SAD_dict = dict(zip(M_G_SAD_list, [[] for i in range(len(M_G_SAD_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        
        placement = game_session[(game_session['event_code']==4020)].correct_flag.values
        correct_placement = placement.sum()
        incorrect_placement = len(placement) - correct_placement 
        
        per_correct_placement  = correct_placement/ (len(placement) + 0.01)
        count_placement = len(placement)
        
        levels_completed = (game_session['event_code']==2050).sum()
        clicked_pic = levels_completed // 3
                
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        M_G_SAD_dict['correct_placement'].append(correct_placement)
        M_G_SAD_dict['incorrect_placement'].append(incorrect_placement)
        M_G_SAD_dict['per_correct_placement'].append(per_correct_placement)
        M_G_SAD_dict['count_placement'].append(count_placement)
        M_G_SAD_dict['levels_completed'].append(levels_completed)
        M_G_SAD_dict['clicked_pic'].append(clicked_pic)
        
        M_G_SAD_dict['extra_clicks'].append(extra_clicks)
        M_G_SAD_dict['help_count'].append(help_count)
        M_G_SAD_dict['game_time'].append(game_time)
        M_G_SAD_dict['event_count'].append(event_count)
        M_G_SAD_dict['speed'].append(speed)
    
    # get final aggregate features
    M_G_SAD_agg_dict = {}
    [M_G_SAD_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in M_G_SAD_dict.items()]
    
    return M_G_SAD_agg_dict

def get_M_G_DDR_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    M_G_DDR_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "skip_tutorial", "rounds",\
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    M_G_DDR_dict = dict(zip(M_G_DDR_list, [[] for i in range(len(M_G_DDR_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        correct_placement = (game_session['event_code']==3021).sum()
        incorrect_placement = (game_session['event_code']==3020).sum()
        
        per_correct_placement  = correct_placement/ (correct_placement + incorrect_placement + 0.01)
        count_placement = correct_placement + incorrect_placement
        
        skip_tutorial = (game_session['event_code']==2075).sum()
        rounds = (game_session['event_code']==2030).sum()

        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        M_G_DDR_dict['correct_placement'].append(correct_placement)
        M_G_DDR_dict['incorrect_placement'].append(incorrect_placement)
        M_G_DDR_dict['per_correct_placement'].append(per_correct_placement)
        M_G_DDR_dict['count_placement'].append(count_placement)
        M_G_DDR_dict['skip_tutorial'].append(skip_tutorial)
        M_G_DDR_dict['rounds'].append(rounds)
        
        M_G_DDR_dict['extra_clicks'].append(extra_clicks)
        M_G_DDR_dict['help_count'].append(help_count)
        M_G_DDR_dict['game_time'].append(game_time)
        M_G_DDR_dict['event_count'].append(event_count)
        M_G_DDR_dict['speed'].append(speed)
    
    # get final aggregate features
    M_G_DDR_agg_dict = {}
    [M_G_DDR_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in M_G_DDR_dict.items()]
    
    return M_G_DDR_agg_dict

def get_M_G_BB_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    M_G_BB_list = ["correct_guess", "incorrect_guess", "per_correct_guess", "count_guess", \
                   "interactive_clicks", "drag_toy", \
                   "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    M_G_BB_dict = dict(zip(M_G_BB_list, [[] for i in range(len(M_G_BB_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        guess = game_session[(game_session['event_code']==4020)].correct_flag.values
        correct_guess = guess.sum()
        incorrect_guess = len(guess) - correct_guess 

        per_correct_guess  = correct_guess/ (len(guess) + 0.01)
        count_guess = len(guess)
        
        interactive_clicks = (game_session['event_code']==4220).sum()
        drag_toy = (game_session['event_code']==4230).sum()
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        M_G_BB_dict['correct_guess'].append(correct_guess)
        M_G_BB_dict['incorrect_guess'].append(incorrect_guess)
        M_G_BB_dict['per_correct_guess'].append(per_correct_guess)
        M_G_BB_dict['count_guess'].append(count_guess)
        M_G_BB_dict['interactive_clicks'].append(interactive_clicks)
        M_G_BB_dict['drag_toy'].append(drag_toy)
        
        M_G_BB_dict['extra_clicks'].append(extra_clicks)
        M_G_BB_dict['help_count'].append(help_count)
        M_G_BB_dict['game_time'].append(game_time)
        M_G_BB_dict['event_count'].append(event_count)
        M_G_BB_dict['speed'].append(speed)
    
    # get final aggregate features
    M_G_BB_agg_dict = {}
    [M_G_BB_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in M_G_BB_dict.items()]
    
    return M_G_BB_agg_dict

def get_M_G_DD_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    M_G_DD_list = ["correct_tap", "incorrect_tap", "per_correct_tap", "count_tap", "rounds_played",  \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    M_G_DD_dict = dict(zip(M_G_DD_list, [[] for i in range(len(M_G_DD_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
        
        tap = game_session[(game_session['event_code']==4020)].correct_flag.values
        correct_tap = tap.sum()
        incorrect_tap = len(tap) - correct_tap 
        
        per_correct_tap  = correct_tap/ (len(tap) + 0.01)
        count_tap = len(tap)
        
        rounds_played = (game_session['event_code']==2020).sum()

        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        M_G_DD_dict['correct_tap'].append(correct_tap)
        M_G_DD_dict['incorrect_tap'].append(incorrect_tap)
        M_G_DD_dict['rounds_played'].append(rounds_played)
        M_G_DD_dict['per_correct_tap'].append(per_correct_tap)
        M_G_DD_dict['count_tap'].append(count_tap)

        M_G_DD_dict['extra_clicks'].append(extra_clicks)
        M_G_DD_dict['help_count'].append(help_count)
        M_G_DD_dict['game_time'].append(game_time)
        M_G_DD_dict['event_count'].append(event_count)
        M_G_DD_dict['speed'].append(speed)
    
    # get final aggregate features
    M_G_DD_agg_dict = {}
    [M_G_DD_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in M_G_DD_dict.items()]
    
    return M_G_DD_agg_dict

def get_T_G_ASS_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    T_G_ASS_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                    "rounds_played", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    T_G_ASS_dict = dict(zip(T_G_ASS_list, [[] for i in range(len(T_G_ASS_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        placement = game_session[(game_session['event_code']==4020)].correct_flag.values
        correct_placement = placement.sum()
        incorrect_placement = len(placement) - correct_placement
        
        per_correct_placement  = correct_placement/ (len(placement) + 0.01)
        count_placement = len(placement)

        rounds_played = (game_session['event_code']==2020).sum()
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        T_G_ASS_dict['correct_placement'].append(correct_placement)
        T_G_ASS_dict['incorrect_placement'].append(incorrect_placement)
        T_G_ASS_dict['per_correct_placement'].append(per_correct_placement)
        T_G_ASS_dict['count_placement'].append(count_placement)
        T_G_ASS_dict['rounds_played'].append(rounds_played)
        
        T_G_ASS_dict['extra_clicks'].append(extra_clicks)
        T_G_ASS_dict['help_count'].append(help_count)
        T_G_ASS_dict['game_time'].append(game_time)
        T_G_ASS_dict['event_count'].append(event_count)
        T_G_ASS_dict['speed'].append(speed)
    
    # get final aggregate features
    T_G_ASS_agg_dict = {}
    [T_G_ASS_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in T_G_ASS_dict.items()]
    
    return T_G_ASS_agg_dict

def get_T_G_AS_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    T_G_AS_list = ["correct_placement", "incorrect_placement", "per_correct_placement", "count_placement", \
                   "win", "miss", "per_win", "count_result", "skip_tutorial", "tutorials_watched", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    T_G_AS_dict = dict(zip(T_G_AS_list, [[] for i in range(len(T_G_AS_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        placement = game_session[(game_session['event_code']==4020)].correct_flag.values
        correct_placement = placement.sum()
        incorrect_placement = len(placement) - correct_placement 

        per_correct_placement  = correct_placement/ (len(placement) + 0.01)
        count_placement = len(placement)

        result = game_session[(game_session['event_code']==4100)].correct_flag.values
        win = result.sum()
        miss = len(result)-win      
        
        per_win = win/(len(result) + 0.01)
        count_result = len(result)
    
        skip_tutorial = (game_session['event_code']==2075).sum()
        tutorials_watched = (game_session['event_code']==2070).sum()
                
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        T_G_AS_dict['correct_placement'].append(correct_placement)
        T_G_AS_dict['incorrect_placement'].append(incorrect_placement)
        T_G_AS_dict['per_correct_placement'].append(per_correct_placement)
        T_G_AS_dict['count_placement'].append(count_placement)
        T_G_AS_dict['win'].append(win)
        T_G_AS_dict['miss'].append(miss)
        T_G_AS_dict['per_win'].append(per_win)
        T_G_AS_dict['count_result'].append(count_result)
        T_G_AS_dict['skip_tutorial'].append(skip_tutorial)
        T_G_AS_dict['tutorials_watched'].append(tutorials_watched)
        
        T_G_AS_dict['extra_clicks'].append(extra_clicks)
        T_G_AS_dict['help_count'].append(help_count)
        T_G_AS_dict['game_time'].append(game_time)
        T_G_AS_dict['event_count'].append(event_count)
        T_G_AS_dict['speed'].append(speed)
        
    
    # get final aggregate features
    T_G_AS_agg_dict = {}
    [T_G_AS_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in T_G_AS_dict.items()]
    
    return T_G_AS_agg_dict

def get_T_G_CR_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    T_G_CR_list = ["correct_count", "incorrect_count", "per_correct_count", "count_count", "drag_object", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    T_G_CR_dict = dict(zip(T_G_CR_list, [[] for i in range(len(T_G_CR_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        count = game_session[(game_session['event_code']==4020)].correct_flag.values
        correct_count = count.sum()
        incorrect_count = len(count) - correct_count 

        per_correct_count  = correct_count/ (len(count) + 0.01)
        count_count = len(count)

        drag_object = (game_session['event_code']==4050).sum()
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        T_G_CR_dict['correct_count'].append(correct_count)
        T_G_CR_dict['incorrect_count'].append(incorrect_count)
        T_G_CR_dict['per_correct_count'].append(per_correct_count)
        T_G_CR_dict['count_count'].append(count_count)
        T_G_CR_dict['drag_object'].append(drag_object)

        T_G_CR_dict['extra_clicks'].append(extra_clicks)
        T_G_CR_dict['help_count'].append(help_count)
        T_G_CR_dict['game_time'].append(game_time)
        T_G_CR_dict['event_count'].append(event_count)
        T_G_CR_dict['speed'].append(speed)
    
    # get final aggregate features
    T_G_CR_agg_dict = {}
    [T_G_CR_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in T_G_CR_dict.items()]
    
    return T_G_CR_agg_dict

def get_C_G_CT_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    C_G_CT_list = ["correct_placement", "incorrect_pacement", "per_correct_placement", "count_placement", "rounds", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    C_G_CT_dict = dict(zip(C_G_CT_list, [[] for i in range(len(C_G_CT_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        
        placement = game_session[(game_session['event_code']==4020)].correct_flag.values
        correct_placement = placement.sum()
        incorrect_pacement = len(placement) - correct_placement 
        
        per_correct_placement  = correct_placement/ (len(placement) + 0.01)
        count_placement = len(placement)

        rounds = (game_session['event_code']==2020).sum()
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        C_G_CT_dict['correct_placement'].append(correct_placement)
        C_G_CT_dict['incorrect_pacement'].append(incorrect_pacement)
        C_G_CT_dict['per_correct_placement'].append(per_correct_placement)
        C_G_CT_dict['count_placement'].append(count_placement)
        
        C_G_CT_dict['rounds'].append(rounds)
        
        C_G_CT_dict['extra_clicks'].append(extra_clicks)
        C_G_CT_dict['help_count'].append(help_count)
        C_G_CT_dict['game_time'].append(game_time)
        C_G_CT_dict['event_count'].append(event_count)
        C_G_CT_dict['speed'].append(speed)
        
    # get final aggregate features
    C_G_CT_agg_dict = {}
    [C_G_CT_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in C_G_CT_dict.items()]
    
    return C_G_CT_agg_dict

def get_C_G_HC_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    C_G_HC_list = ["win", "miss", "per_win", "count_result", "rounds", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    C_G_HC_dict = dict(zip(C_G_HC_list, [[] for i in range(len(C_G_HC_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        result = game_session[(game_session['event_code']==4020)].correct_flag.values
        win = result.sum()
        miss = len(result)-win   
        
        per_win = win/(len(result) + 0.01)
        count_result = len(result)
        
        rounds = (game_session['event_code']==2030).sum()

        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        C_G_HC_dict['win'].append(win)
        C_G_HC_dict['miss'].append(miss)
        C_G_HC_dict['per_win'].append(per_win)
        C_G_HC_dict['count_result'].append(count_result)
        C_G_HC_dict['rounds'].append(rounds)
        
        C_G_HC_dict['extra_clicks'].append(extra_clicks)
        C_G_HC_dict['help_count'].append(help_count)
        C_G_HC_dict['game_time'].append(game_time)
        C_G_HC_dict['event_count'].append(event_count)
        C_G_HC_dict['speed'].append(speed)
    
    # get final aggregate features
    C_G_HC_agg_dict = {}
    [C_G_HC_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in C_G_HC_dict.items()]
    
    return C_G_HC_agg_dict

def get_C_G_LL_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    C_G_LL_list = ["add", "remove", "rounds", "per_add", "count_placement", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    C_G_LL_dict = dict(zip(C_G_LL_list, [[] for i in range(len(C_G_LL_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        placement = game_session[(game_session['event_code']==4020)].correct_flag.values
        add = placement.sum()
        remove = len(placement) - add 
        
        per_add = add/(len(placement) + 0.01)
        count_placement = len(placement)
        
        rounds = (game_session['event_code']==2030).sum()
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        C_G_LL_dict['add'].append(add)
        C_G_LL_dict['remove'].append(remove)
        C_G_LL_dict['per_add'].append(per_add)
        C_G_LL_dict['count_placement'].append(count_placement)
        C_G_LL_dict['rounds'].append(rounds)
        
        C_G_LL_dict['extra_clicks'].append(extra_clicks)
        C_G_LL_dict['help_count'].append(help_count)
        C_G_LL_dict['game_time'].append(game_time)
        C_G_LL_dict['event_count'].append(event_count)
        C_G_LL_dict['speed'].append(speed)
    
    # get final aggregate features
    C_G_LL_agg_dict = {}
    [C_G_LL_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in C_G_LL_dict.items()]
    
    return C_G_LL_agg_dict

def get_C_G_PB_features(title_session):
    
    title_name = title_session.iloc[0].title
    
#     Initiate raw dict
    C_G_PB_list = ["correct_balance", "incorrect_balance", "per_correct_balance", "count_balance", \
                    "extra_clicks", "help_count", "game_time", "event_count", "speed" \
                   ]
    C_G_PB_dict = dict(zip(C_G_PB_list, [[] for i in range(len(C_G_PB_list))]))
    
    for game, game_session in title_session.groupby('game_session', sort=False):
          
        balance = game_session[(game_session['event_code']==4100)].correct_flag.values
        correct_balance = balance.sum()
        incorrect_balance = len(balance) - correct_balance 
        
        per_correct_balance = correct_balance/(len(balance) + 0.01)
        count_balance = len(balance)
        
        extra_clicks = (game_session['event_code']==4070).sum()
        help_count = (game_session['event_code']==4090).sum()

        game_time = game_session.iloc[-1]['game_time']
        event_count = game_session.iloc[-1]['event_count']
        
        speed = game_time / event_count
        
        C_G_PB_dict['correct_balance'].append(correct_balance)
        C_G_PB_dict['incorrect_balance'].append(incorrect_balance)
        C_G_PB_dict['per_correct_balance'].append(per_correct_balance)
        C_G_PB_dict['count_balance'].append(count_balance)
        
        C_G_PB_dict['extra_clicks'].append(extra_clicks)
        C_G_PB_dict['help_count'].append(help_count)
        C_G_PB_dict['game_time'].append(game_time)
        C_G_PB_dict['event_count'].append(event_count)
        C_G_PB_dict['speed'].append(speed)
    
    # get final aggregate features
    C_G_PB_agg_dict = {}
    [C_G_PB_agg_dict.update({title_name+'_mean_'+key : np.round(np.mean(value),2),  \
                              title_name+'_mode_'+key : mode(value)[0][0], \
                              title_name+'_std_' +key : np.round(np.std(value), 2)}) \
                            for key, value in C_G_PB_dict.items()]
    
    return C_G_PB_agg_dict

# count features

def get_count_features(instance):
    
    count_features_dict = {}
    
    # aggregate on event count
    count_df = instance[(instance['event_count'] == 1)].groupby('title').agg({'game_session':'count'})
    [count_features_dict.update({index+'_count': count_df.loc[index][0]}) for index in count_df.index]

    
    return count_features_dict

In [ ]:
# factory titles
def get_title_features(title_session):
    
    # Assessment titles
    
    if (title_session['title'].iloc[0] == 'T_AS_MS'):
        T_AS_MS_features = get_T_AS_MS_features(title_session)
        return T_AS_MS_features
    
    elif (title_session['title'].iloc[0] == 'T_AS_BM'):
        T_AS_BM_features = get_T_AS_BM_features(title_session)
        return T_AS_BM_features
            
    elif (title_session['title'].iloc[0] == 'C_AS_CB'):
        C_AS_CB_features = get_C_AS_CB_features(title_session)
        return C_AS_CB_features
    
    elif (title_session['title'].iloc[0] == 'C_AS_CS'):
        C_AS_CS_features = get_C_AS_CS_features(title_session)
        return C_AS_CS_features
    
    elif (title_session['title'].iloc[0] == 'M_AS_CF'):
        M_AS_CF_features = get_M_AS_CF_features(title_session)
        return M_AS_CF_features
    
    # Activity titles
    
    elif (title_session['title'].iloc[0] == 'M_AC_SB'):
        M_AC_SB_features = get_M_AC_SB_features(title_session)
        return M_AC_SB_features
    
    elif (title_session['title'].iloc[0] == 'M_AC_WH'):
        M_AC_WH_features = get_M_AC_WH_features(title_session)
        return M_AC_WH_features
            
    elif (title_session['title'].iloc[0] == 'M_AC_BF'):
        M_AC_BF_features = get_M_AC_BF_features(title_session)
        return M_AC_BF_features
    
    elif (title_session['title'].iloc[0] == 'T_AC_BM'):
        T_AC_BM_features = get_T_AC_BM_features(title_session)
        return T_AC_BM_features
    
    elif (title_session['title'].iloc[0] == 'T_AC_FW'):
        T_AC_FW_features = get_T_AC_FW_features(title_session)
        return T_AC_FW_features
    
    elif (title_session['title'].iloc[0] == 'T_AC_F'):
        T_AC_F_features = get_T_AC_F_features(title_session)
        return T_AC_F_features
    
    elif (title_session['title'].iloc[0] == 'C_AC_CB'):
        C_AC_CB_features = get_C_AC_CB_features(title_session)
        return C_AC_CB_features
    
    elif (title_session['title'].iloc[0] == 'C_AC_ED'):
        C_AC_ED_features = get_C_AC_ED_features(title_session)
        return C_AC_ED_features

    # Games titles
    
    elif (title_session['title'].iloc[0] == 'M_G_SAD'):
        M_G_SAD_features = get_M_G_SAD_features(title_session)
        return M_G_SAD_features
    
    elif (title_session['title'].iloc[0] == 'M_G_DDR'):
        M_G_DDR_features = get_M_G_DDR_features(title_session)
        return M_G_DDR_features
            
    elif (title_session['title'].iloc[0] == 'M_G_BB'):
        M_G_BB_features = get_M_G_BB_features(title_session)
        return M_G_BB_features
    
    elif (title_session['title'].iloc[0] == 'M_G_DD'):
        M_G_DD_features = get_M_G_DD_features(title_session)
        return M_G_DD_features
    
    elif (title_session['title'].iloc[0] == 'T_G_ASS'):
        T_G_ASS_features = get_T_G_ASS_features(title_session)
        return T_G_ASS_features
    
    elif (title_session['title'].iloc[0] == 'T_G_AS'):
        T_G_AS_features = get_T_G_AS_features(title_session)
        return T_G_AS_features
    
    elif (title_session['title'].iloc[0] == 'T_G_CR'):
        T_G_CR_features = get_T_G_CR_features(title_session)
        return T_G_CR_features
    
    elif (title_session['title'].iloc[0] == 'C_G_CT'):
        C_G_CT_features = get_C_G_CT_features(title_session)
        return C_G_CT_features
    
    elif (title_session['title'].iloc[0] == 'C_G_HC'):
        C_G_HC_features = get_C_G_HC_features(title_session)
        return C_G_HC_features
    
    elif (title_session['title'].iloc[0] == 'C_G_LL'):
        C_G_LL_features = get_C_G_LL_features(title_session)
        return C_G_LL_features
    
    elif (title_session['title'].iloc[0] == 'C_G_PB'):
        C_G_PB_features = get_C_G_PB_features(title_session)
        return C_G_PB_features

    
    return 'ToDo'

In [ ]:
# extract timestamp features
def extract_time_features(df):
#     df['timestamp'] = pd.to_datetime(df['timestamp'])
#     df['date'] = df['timestamp'].dt.date
    df.loc[:,'month'] = df['timestamp'].dt.month
    df.loc[:,'hour'] = df['timestamp'].dt.hour
#     df['year'] = df['timestamp'].dt.year
    df.loc[:,'dayofweek'] = df['timestamp'].dt.dayofweek
    df.loc[:,'weekofyear'] = df['timestamp'].dt.weekofyear
    df.loc[:,'dayofyear'] = df['timestamp'].dt.dayofyear
    df.loc[:,'quarter'] = df['timestamp'].dt.quarter
    df.loc[:,'is_month_start'] = df['timestamp'].dt.is_month_start.apply(int) 
    df['is_weekend']=np.where(((df['timestamp'].dt.day_name()=='Sunday')|(df['timestamp'].dt.day_name()=='Saturday')),1,0)


    return df

In [ ]:
def get_features(data, train_labels=None, test_set=False):
    print(len(data))
       
    # Correct the timestamp type
    data['timestamp'] = pd.to_datetime(data['timestamp'])
#     data['title'] = data['title'].map(titles_dict)
    
    if test_set == False:
        # Join with train_labels to extract accuracy and know which assessment is valid for training data
        valid_train = data.merge(train_labels, on=['installation_id', 'game_session'], how='left') \
                       .drop(['title_y', 'num_correct', 'num_incorrect', 'accuracy'], axis=1)
        valid_train.rename(columns = {'title_x':'title'}, inplace = True)
        
        # create train dataframe with groupby on installationID
        data_groupby_installationID = valid_train.groupby('installation_id', sort=False)
    else:
        # create test dataframe with groupby on installationID
        data_groupby_installationID = data.groupby(['installation_id'])
        
    #====================================================================#  
    # initialise all the features dictionary
    raw_features_dict = generate_final_raw_features_dict()

    #====================================================================#
    
    # Initialiazing the list which will contain all the training instance
    compiled_data = []
    
    # assign value to total which will be helpfull while looking at the progress bar 
    if test_set == True:
        total = 1000
    else:
        total = 17000
    
    # Loop over every intallationID to extract features
    for i, (ins_id, user_sample) in tqdm(enumerate(data_groupby_installationID), total= total):
        # sort user_sample by time
        user_sample = user_sample.sort_values(by='timestamp')
        
        if test_set == False: 
            # extract assessments which are valid for thraining data
            valid_assessment = user_sample[(user_sample['type'] == 'Assessment') \
                                     & (user_sample['event_count'] == 1) \
                                     & (user_sample['accuracy_group'].notnull())]

            # Loop over every valid assessement for each installationID and 
            # generate training feature instance.
            for index, row in valid_assessment.iterrows():

                accuracy_group = row["accuracy_group"]
                timestamp = row["timestamp"]
                title = row["title"]

                # Extract user history before valid assessment
                instance = user_sample[user_sample['timestamp'] < timestamp]

                # Initiate instance feature dictionary
                feature_dict = raw_features_dict.copy()
#                 # extract title features
                detailed_features = instance.groupby('title').apply(get_title_features)
                [feature_dict.update(item) for item in detailed_features.values if type(item) == dict]
                # extract title counts
                count_features_dict = get_count_features(instance)
                feature_dict.update(count_features_dict)
                
                feature_dict['title'] = title
                
                # Add Label (accuracy_group)
                feature_dict['accuracy_group'] = accuracy_group
                

                # add time components 
                feature_dict['timestamp'] = timestamp
        
                compiled_data.append(feature_dict.copy())
        
        else:
                installation_id = user_sample.iloc[-1]['installation_id']
                timestamp = user_sample.iloc[-1]['timestamp']
                title = user_sample.iloc[-1]['title']
                    
                # Extract user history before assessment    
                instance = user_sample[:-1]
            
                # Initiate instance feature dictionary
                feature_dict = raw_features_dict.copy()
                
                feature_dict['installation_id'] = installation_id
                
                # extract title features
                detailed_features = instance.groupby('title').apply(get_title_features)
                [feature_dict.update(item) for item in detailed_features.values if type(item) == dict]
                # extract title counts
                count_features_dict = get_count_features(instance)
                feature_dict.update(count_features_dict)

                feature_dict['title'] = title

                # add time components 
                feature_dict['timestamp'] = timestamp

                compiled_data.append(feature_dict.copy())
                        
    # convery list of dictionary into pandas dataframe
    feature_data = pd.DataFrame(compiled_data)
    #extract timestamp features
    extract_time_features(feature_data)
    feature_data.drop(['timestamp'], axis=1, inplace=True)
    
    return feature_data
   

In [ ]:
%%time
train_features = get_features(train, train_labels, test_set=False)   

In [ ]:
%%time
test_features = get_features(test, test_set=True) 

In [ ]:
import pandas as pd
import numpy as np
import os, sys
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn.metrics import confusion_matrix

# import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split as split
from sklearn.metrics import cohen_kappa_score
import category_encoders as ce

# # from catboost import CatBoostRegressor
import catboost as cat
import lightgbm as lgb
import xgboost as xgb

from functools import partial
import scipy as sp              # for optimize.minimize()

In [ ]:
class OptRounder(object):
    def __init__(self):
        self.res_ = []
        self.coef_ = []
        
    def get_res(self):
        return self.res_
    
    # objective function
    def func(self, coef, X, y):
        kappa = cohen_kappa_score(self.bincut(coef, X), y,
                                  weights='quadratic')
        return -kappa

    def bincut(self, coef, X):
        return pd.cut(X,
                      [-np.inf] + list(np.sort(coef)) + [np.inf],
                      labels = [0, 1, 2, 3])
        
    def fit(self, X, y):
        pfunc = partial(self.func, X=X, y=y)
        self.res_ = sp.optimize.minimize(fun = pfunc,           # objective func
                                         x0 = [0.6, 1.1, 2.02],  # initial coef
                                         method='nelder-mead')  # solver
        self.coef_ = self.res_.x
        
    def predict(self, X, coef):
        return self.bincut(coef, X)

def qwk(act,pred, n=4, hist_range=(0,3)):
    
    O = confusion_matrix(act,pred)
    O = np.divide(O,np.sum(O))
    
    W = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            W[i][j] = ((i-j)**2)/((n-1)**2)
            
    act_hist = np.histogram(act,bins=n,range=hist_range)[0]
    prd_hist = np.histogram(pred,bins=n,range=hist_range)[0]
    
    E = np.outer(act_hist,prd_hist)
    E = np.divide(E,np.sum(E))
    
    num = np.sum(np.multiply(W,O))
    den = np.sum(np.multiply(W,E))
        
    return 1-np.divide(num,den)

In [ ]:
mode_acc_features = [x for x in train_features.columns if 'mode_acc' in x]
without_mode = [x for x in train_features.columns if 'mode' not in x]
without_mode_help_count = [x for x in without_mode if 'help_count' not in x]

In [ ]:
time_features = ['hour', 'dayofweek', 'is_weekend']
acc_features = [x for x in without_mode_help_count if '_acc' in x]
per_features = [x for x in without_mode_help_count if 'per_' in x]
count_features = [x for x in without_mode_help_count if 'count' in x]
speed_features = [x for x in without_mode_help_count if 'speed' in x]

extra_features = [x for x in without_mode_help_count if 'extra_clicks' in x]

In [ ]:
all_features = list(set(['title'] + time_features + acc_features + per_features + 
                        count_features + speed_features + extra_features + mode_acc_features))

categorical_features = ['title', 'hour', 'dayofweek']


In [ ]:
# train_features = train_features[[x for x in train_features.columns if 'mode' not in x]]
# train_features = train_features[[x for x in train_features.columns if 'help_count' not in x]]

In [ ]:
# time_features = ['hour', 'dayofweek', 'is_weekend']
# acc_features = [x for x in train_features.columns if '_acc' in x]
# per_features = [x for x in train_features.columns if 'per_' in x]
# count_features = [x for x in train_features.columns if 'count' in x]
# speed_features = [x for x in train_features.columns if 'speed' in x]

# extra_features = [x for x in train_features.columns if 'extra_clicks' in x]

In [ ]:
# all_features = list(set(['title'] + time_features + acc_features + per_features + 
#                         count_features + speed_features + extra_features))

# categorical_features = ['title', 'hour', 'dayofweek']

In [ ]:
# dataset
X, y = train_features[all_features], train_features['accuracy_group']

X_test = test_features[all_features]

In [ ]:
# Create multiple datasets to create multiple models (not for CV).
NFOLDS = 15
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)

In [ ]:
# CatBoost
start_time = time()

cat_models = []

cat_params = {
    'learning_rate': 0.01,
    'loss_function': 'RMSE',
    'random_seed': 42,
    'depth': 8,                            # add
    'border_count': 108,                    # add
    'bagging_temperature': 3,
    'one_hot_max_size' : 5,
    'l2_leaf_reg' : 9,
}

coefficients = [1.09669537, 1.7449929,  2.19126902]
optR = OptRounder()

preds = []
cat_oof = np.zeros(len(y))

for fold, (train_ids, val_ids) in enumerate(folds.split(X,y)):
    print('● Fold :', fold+1,'/',NFOLDS)
    
    dtrain = cat.Pool(X.iloc[train_ids], y[train_ids], cat_features=categorical_features)
    dval = cat.Pool(X.iloc[val_ids], y[val_ids], cat_features=categorical_features)
    cat_model = cat.train(params=cat_params,
                      dtrain=dtrain,
                      eval_set=dval,        
                      iterations=5000,      
                      early_stopping_rounds=100,
                      verbose=200
                     )
    cat_models.append(cat_model)
    
    cat_pred = cat_model.predict(X.iloc[val_ids])
    preds.append(cat_pred)
    
    cat_oof[val_ids] = cat_pred
    
    df2 = pd.DataFrame(cat_pred)    
    # Calculate the average value of each model pred
    df2['mean'] = df2.mean(axis = 'columns')

    df2['predict'] = optR.predict(df2['mean'].values, coefficients).astype(int)
    df2['y'] = y[val_ids].values.astype(int)
    
    print('cat qwk score: ', qwk(df2['y'], df2['predict']))

y_pred = optR.predict(cat_oof, coefficients).astype(int)
print('cat total qwk score: ', qwk(y_pred, y))    
print('\nTime:', time() - start_time)

In [ ]:
preds = []
# CatBoost models
for model in cat_models:
    pred = model.predict(X)
    
    optR = OptRounder()
    optR.fit(pred, y)
    coefficients = optR.get_res().x
    
    pred = model.predict(X_test)
    y_hat = optR.predict(pred, coefficients).astype(int)
    
    preds.append(y_hat)

In [ ]:
from scipy import stats

df = pd.DataFrame(preds).T

a = df.values.T
b = stats.mode(a)

df['pred'] = b[0][0]

In [ ]:
submission['accuracy_group'] = df['pred']
submission.to_csv('submission.csv', index=None)
submission.head()